In [201]:
'''
The script is exploring each of the questions asked in the assignment in detail, and answering them as best as possible.

The dataset studied was supplied by Vlammer is stored in the current directory with the name Titanic_data.xlsx.

Analysis uses python libraries such as pandas, [matplotlib, seaborn, and numpy]
'''

import pandas as pd
import numpy as np
import re # For using split function with multiple delimiters 

In [2]:
titanic_original = pd.read_excel(r'Titanic_data.xlsx')

In [3]:
titanic_original.head()

,PassengerId,Survived2,Pclass,Name,Sex,Age,SibSp,Ticket,Fare,Cabin,Embarked
0,1,Yes,3,"Braund, Mr. Owen Harris",male,22.0,1,A/5 21171,725,NaN,Cherbourg
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,PC 17599,712833,C85,Queenstown
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,STON/O2. 3101282,7925,NaN,Cherbourg
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,113803,531,C123,Cherbourg
4,5,Yes,3,"Allen, Mr. William Henry",male,35.0,0,373450,805,NaN,Cherbourg


1. What do you think about the quality of the data?


In [14]:
#Code towards the answer
# Total number of columns, and number of entries.
print(titanic_original.shape)
# What are the different columns, and their type?
print(titanic_original.info())
# How many of them entries are null or missing?
print(titanic_original.isna().sum())

(891, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived2    891 non-null    object 
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    int64  
 9   Cabin        204 non-null    object 
 10  Embarked     891 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 76.7+ KB
None
PassengerId      0
Survived2        0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


In [28]:
# Different values under columns
columns = list(titanic_original.columns)
columns.remove('Name')
for column in columns[1:]:
    print(titanic_original[column].value_counts(),'\n')

Yes    549
0      342
Name: Survived2, dtype: int64 

3    491
1    216
2    184
Name: Pclass, dtype: int64 

male      577
female    314
Name: Sex, dtype: int64 

24.0     30
22.0     27
18.0     26
28.0     25
19.0     25
         ..
74.0      1
67.0      1
80.0      1
66.0      1
345.0     1
Name: Age, Length: 87, dtype: int64 

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64 

CA. 2343              7
347082                7
1601                  7
3101295               6
347088                6
                     ..
112277                1
112059                1
SOTON/O.Q. 3101306    1
244358                1
2663                  1
Name: Ticket, Length: 681, dtype: int64 

805      43
13       42
78958    38
775      34
26       31
         ..
86542     1
22025     1
12525     1
76292     1
22525     1
Name: Fare, Length: 246, dtype: int64 

B96 B98        4
G6             4
C23 C25 C27    4
F2             3
F33            3
            

The dataset supplied came without a data dictionary, and therefore was compiled for further use and analysis. The data dictionary looks at the inherent quality of the data. This is based on the guidelines mentioned in [wikipedia](https://en.wikipedia.org/wiki/Data_cleansing#Data_quality) and advised in this [tutorial](https://medium.com/better-programming/data-cleaning-with-python-pandas-an-introduction-1cfd5cde6884). Additionally that data dictionary also gives description of how it is modified further for easier and comprehensive analysis of the underlying information. In the data dictionary, description of the engineered columns are also provided, however this is added at a later stage of exploratory analysis.

The following points are investigated in regard to checking the quality of data.
* Data validity
    * Is the datatype correct for the corresponding column values?
    * Is the data value making sense and falling within a range?
* Missing data
    * Is any of the columns missing data?
* Duplicates
    * Are there duplicate entries or duplicated within columns with unique values, such as passengerID, tickets and name. It is possible for two people to have similar names, but very unlikely on the titanic to share their entire name.
* Uniformity 
    * Are the values within a column scaled differenty? Such as price of some passengers in pounds, and others in dollars? Or are the values denoted in different denominations for different passengers? Or that some of the values are in numbers and others in strings?
* Consistency and accuracy
    * Since the titanic dataset is a popular dataset in the data science community, and as well as information of all passengers being readily available online. I used it to verify the information presented to that is open source noting for any inaccuracies or inconsistencies. However this is done with caution as this analysis being part of a job selection procedure, the underlying data may be expected to preserved keenly as required for the interviewing the specific sections of the data.


In [95]:
# Checking the type of data in each of the columns
titanic_original.dtypes

PassengerId      int64
Survived2       object
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Ticket          object
Fare             int64
Cabin           object
Embarked        object
dtype: object

The datatypes seem to be alright, age is noticed to be float64 instead of int64. Does it imply that it holds fractions?

In [58]:
print(titanic_original.Age.apply(float.is_integer).value_counts())
print(titanic_original.Age.isna().value_counts())

True     714
False    177
Name: Age, dtype: int64
False    714
True     177
Name: Age, dtype: int64


A quick look shows that even though age is denoted as float, it holds only integers. Therefore it can be converted to int64 without losing information and is converted as it needs less memory than float64.

In [ ]:
# Data dictionary
# Original:

# 11 features, and 891 entries of passengers. 
# Crew information is not included assumption. [This can be verified by checking at the pclass data]

#1st column is passenger id and the primary key for the dataset: This acts as a unique identifier for each of the entry.
#This feature needn't be explored further in our analysis

#2nd column is about the whether the passenger died in the sinking. This is infered from the  
#It is marked with 2 values, Yes for survival and 0 for demise. 

#6th column denotes the Age of the passengers.
#The values are in integers, even though stored as floating point numbers. It can also be seen that the age is missing for 177 entries.


# After cleaning

#Survived2 column is renamed to survived, and all the Yes values are changed 0 for demise, and 0 to 1 for surviving the shipwreck. 
#pclass is renamed to Class
#Age column is converted to int64, with null values marked as 0. 
# After feature engineering

In [112]:
#Investigation of the missing data in Age column
print("Information of people without an age entry:\n")
for column in list(titanic_original.columns)[1:]:
    if column in ['Name', 'Age']:
        continue
    else:
        print(titanic_original[titanic_original.Age.isna()][column].value_counts(),"\n")
#print(titanic_original[titanic_original.Age.isna()].Pclass.value_counts())
#print(titanic_original[titanic_original.Age.isna()].Sex.value_counts())
#print(titanic_original[titanic_original.Age.isna()].SibSp.value_counts())
#print(titanic_original[titanic_original.Age.isna()].SibSp.value_counts())

Information of people without an age entry:

Yes    125
0       52
Name: Survived2, dtype: int64 

3    136
1     30
2     11
Name: Pclass, dtype: int64 

male      124
female     53
Name: Sex, dtype: int64 

0    137
1     26
8      7
3      4
2      3
Name: SibSp, dtype: int64 

CA. 2343    7
4133        4
371110      3
239853      3
1601        3
           ..
PC 17318    1
2649        1
349223      1
36209       1
374910      1
Name: Ticket, Length: 155, dtype: int64 

775      20
78958    15
805      14
0         8
6955      7
         ..
355       1
86625     1
50        1
95        1
87125     1
Name: Fare, Length: 73, dtype: int64 

F38      1
C124     1
A14      1
C52      1
C92      1
C47      1
F E69    1
B102     1
D21      1
C106     1
A19      1
E33      1
E101     1
C128     1
C95      1
C126     1
D45      1
A32      1
B78      1
Name: Cabin, dtype: int64 

Cherbourg       90
Southhampton    49
Queenstown      38
Name: Embarked, dtype: int64 



#### Investigation of null values in the Age section
* Of the entries missing information about, more than 70 percent of them are survivors. This information easily be filled, as the information of them would be available. This is also interesting and begs the question on why these ages were missing in the first place.
* It can also be seen that the of the passengers missing ages, a few of them share tickets or have siblings, if there are survivors who knew these passengers, it would be another way to fill in their ages.
* Majority of the passengers with missing information seem to have boarded from Cherbourg. What was class distribution of passengers boarding from Cherbourg?
* Cabin information of few of these passengers are available.
* Regarding class distribution, majority of passengers with missing information come from 3rd class. However we also few around 30 first class passengers with missing information. Isn't this a little strange, as the people in 1st class were who's who of the time? 
* How well do these information stand, when I run the analysis 1 layer deeper?

In [195]:
print("Information of people without an age entry:\n")
print(titanic_original[(titanic_original.Age.isna())].groupby(["Pclass", "Sex"]).Survived2.value_counts().unstack().fillna(0).sort_values('Yes',ascending=False),"\n")
#print(titanic_original[titanic_original.Age.isna()].value_counts(["Ticket","Survived2"])[:20],"\n")
print(titanic_original[titanic_original.Age.isna()].groupby(["Ticket"]).Survived2.value_counts().unstack().fillna(0).sort_values('Yes',ascending=False),"\n")


Information of people without an age entry:

Survived2         0   Yes
Pclass Sex               
3      male     9.0  85.0
       female  25.0  17.0
1      male     5.0  16.0
2      male     2.0   7.0
1      female   9.0   0.0
2      female   2.0   0.0 

Survived2     0  Yes
Ticket              
CA. 2343    0.0  7.0
4133        0.0  4.0
239853      0.0  3.0
W./C. 6607  0.0  2.0
371110      1.0  2.0
...         ...  ...
9234        1.0  0.0
226593      1.0  0.0
19988       1.0  0.0
19947       1.0  0.0
19996       1.0  0.0

[155 rows x 2 columns] 



* It is interesting to note that of the passengers with missing age, that did survive were more men than female, and they came from the third class. And of the female survivors they all came from third class. Is it also possible the missing age entry is due to non-disclosure by the passengers?

**In the end the main takeway for me is on why survivors age isn't already available?**  
* Could it be because as this dataset was designed as a tutorial for beginning data scientists? 




In [223]:
#Code to check the titles of the passengers and creating a new column called Title to store the titles.
titles = []
for name in titanic_original.Name:
    #print(name)
    name = name.replace(" ", "")
    name = name.replace(".",",")
    #print(name)
    title = re.split(',',name)[1]
    titles.append(title)
titanic_original["Title"] = titles

In [225]:
#This column of titles will be useful in filling in missing values of Age. 
titanic_original.Title.value_counts()

Mr             517
Miss           182
Mrs            125
Master          40
Dr               7
Rev              6
Major            2
Col              2
Mlle             2
Sir              1
Mme              1
theCountess      1
Don              1
Lady             1
Capt             1
Jonkheer         1
Ms               1
Name: Title, dtype: int64

In [232]:
#Investigation of survival of the passengers based on the title.
titanic_original.groupby('Title').Survived2.value_counts().unstack().fillna(0)

Survived2,0,Yes
Title,,
Capt,0.0,1.0
Col,1.0,1.0
Don,0.0,1.0
Dr,3.0,4.0
Jonkheer,0.0,1.0
Lady,1.0,0.0
Major,1.0,1.0
Master,23.0,17.0
Miss,127.0,55.0


**Is the survival state of passengers mixed up?**

In [235]:
print(titanic_original.Survived2.value_counts())
print(titanic_original.groupby('Survived2').Sex.value_counts())

Yes    549
0      342
Name: Survived2, dtype: int64
Survived2  Sex   
0          female    233
           male      109
Yes        male      468
           female     81
Name: Sex, dtype: int64


Aha, that seems to be the case. According to the dataset, there are more survivors than who died, and similarly, more male survivors in comparison. This is due to erraneous description of the data, and is corrected in the cleaned dataset.


In [250]:
#Cleaning the data

#Deep copy of the dataset.
titanic_cleaned = titanic_original.copy()

In [251]:
#Renaming of the survival column
titanic_cleaned.rename(columns={'Survived2':'Survived', 'Pclass':'Class'},inplace=True)
titanic_cleaned['Survived'].replace(to_replace={'Yes':0, 0:1}, inplace=True)
# 
titanic_cleaned.Age = titanic_cleaned.Age.fillna(0.0).astype(np.int64)

In [252]:
titanic_cleaned.Survived = titanic_cleaned.Survived.astype(np.int64)

In [253]:
titanic_cleaned.head()

,PassengerId,Survived,Class,Name,Sex,Age,SibSp,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,A/5 21171,725,NaN,Cherbourg,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,PC 17599,712833,C85,Queenstown,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,STON/O2. 3101282,7925,NaN,Cherbourg,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,113803,531,C123,Cherbourg,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35,0,373450,805,NaN,Cherbourg,Mr


In [90]:
titanic_cleaned.dtypes

PassengerId     int64
Survived        int64
Pclass          int64
Name           object
Sex            object
Age             int64
SibSp           int64
Ticket         object
Fare            int64
Cabin          object
Embarked       object
dtype: object